In [ ]:
from dotenv import load_dotenv
import os
import utils
import pandas as pd
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import SummaryIndex
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    SimpleDirectoryReader,
    load_indices_from_storage,
    load_index_from_storage,
    StorageContext
)

In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('OPENAI_API_BASE')
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

In [3]:
openai_embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    api_key=os.environ["OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

In [4]:
llm = AzureOpenAI(
    engine="stb-gpt-35-turbo", model="gpt-35-turbo", temperature=0.0
    #engine="STB-POC-JUNE", model="gpt-4", temperature=0.0
)

In [5]:
#Settings
Settings.llm = llm
Settings.embed_model = openai_embed_model

In [ ]:
url_list = utils.read_excel_to_list(pd,'data/urls.xlsx')
urls = utils.get_values(url_list)
print(urls)

In [ ]:
# storage_context = StorageContext.from_defaults(
#     docstore=SimpleDocumentStore.from_persist_dir(persist_dir="<persist_dir>"),
#     vector_store=SimpleVectorStore.from_persist_dir(persist_dir="<persist_dir>"),
#     index_store=SimpleIndexStore.from_persist_dir(persist_dir="<persist_dir>"),
# )

In [ ]:
documents = SimpleWebPageReader(html_to_text=True).load_data(urls)
index_web = VectorStoreIndex.from_documents(documents)

In [ ]:
documents = SimpleDirectoryReader('data/docs').load_data()
index_doc = VectorStoreIndex.from_documents(documents,show_progress=True)

In [ ]:
# load index from disk
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="storage1")
# load index
index = load_index_from_storage(storage_context, index_id=["index_web","index_doc"])
print(index)